# Desafio 1 - Pedro Lucas Barrera a1801

In [1]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

In [2]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

In [3]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
tfidfvect = TfidfVectorizer()
X_train = tfidfvect.fit_transform(newsgroups_train.data)

## Punto 1: 
 Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos. Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido la similaridad según el contenido del texto y la etiqueta de clasificación.

In [ ]:
#Tomamos 5 documentos al azar
np.random.seed(42)  # Para reproducibilidad
indices = np.random.choice(X_train.shape[0], 5, replace=False)
random_items = X_train[indices]


In [ ]:
print(f"Total de documentos en el dataset: {X_train.shape[0]}")
print(f"Dimensionalidad del vector TF-IDF: {X_train.shape[1]}")
print(f"Índices de documentos seleccionados: {indices}\n")

# Para cada documento seleccionado, busco los 5 más similares
indexes_most_similar_documents = {}
for i, doc_index in enumerate(indices):
    print(f"DOCUMENTO {i+1} (numero: {doc_index})")
    
    
    current_doc = X_train[doc_index]
    similarities = cosine_similarity(current_doc, X_train).flatten()
    
    #Tomamos 6 elementos con mayor simulitud, dado que el primero sera el mismo documento (con similaridad = 1.0), y lo removemos el primero.
    most_similar_indices = similarities.argsort()[-6:][::-1]
    most_similar_indices = most_similar_indices[1:]
    indexes_most_similar_documents[i] = most_similar_indices
    
    print(f"Etiqueta del documento a comprar: {newsgroups_train.target_names[newsgroups_train.target[doc_index]]}")
    print("LOS 5 DOCUMENTOS MÁS SIMILARES:")
    
    for rank, similar_index in enumerate(most_similar_indices, 1):
        similarity_score = similarities[similar_index]
        similar_label = newsgroups_train.target_names[newsgroups_train.target[similar_index]]
        
        print(f"### {rank}°: Documento numero {similar_index}")
        print(f"- Similaridad: {similarity_score:.4f}")
        print(f"- Etiqueta: {similar_label}")
    print("\n------------------------------------------------------------------------------------")


Total de documentos en el dataset: 11314
Dimensionalidad del vector TF-IDF: 101631
Índices de documentos seleccionados: [7492 3546 5582 4793 3813]

DOCUMENTO 1 (numero: 7492)
Etiqueta del documento a comprar: comp.sys.mac.hardware
LOS 5 DOCUMENTOS MÁS SIMILARES:
### 1°: Documento numero 10935
- Similaridad: 0.6665
- Etiqueta: comp.sys.mac.hardware
### 2°: Documento numero 7258
- Similaridad: 0.3476
- Etiqueta: comp.sys.ibm.pc.hardware
### 3°: Documento numero 4971
- Similaridad: 0.1799
- Etiqueta: comp.sys.mac.hardware
### 4°: Documento numero 4303
- Similaridad: 0.1547
- Etiqueta: misc.forsale
### 5°: Documento numero 645
- Similaridad: 0.1414
- Etiqueta: comp.sys.mac.hardware

------------------------------------------------------------------------------------
DOCUMENTO 2 (numero: 3546)
Etiqueta del documento a comprar: comp.os.ms-windows.misc
LOS 5 DOCUMENTOS MÁS SIMILARES:
### 1°: Documento numero 5665
- Similaridad: 0.2040
- Etiqueta: comp.sys.ibm.pc.hardware
### 2°: Documento num

### Analizo la similitud tomando en cuenta su titulo y contenido:
- Documento1: 
    Casi todos los titulos de los documentos mas similares son de computacion, al igual con el que se lo compara. Sin embargo, hay uno que es 'misc.forsale', osea, miscelaneo a la venta. Viendo el contenido del mismo, puede verse que....

Imprimo los documentos mas similares por cada uno de los elegidos al azar y los comparo:

## Punto 2: 
  Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación (f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial y ComplementNB.

In [18]:
X_test = tfidfvect.transform(newsgroups_test.data)
y_train = newsgroups_train.target
y_test = newsgroups_test.target

print(f"Forma del conjunto de entrenamiento: {X_train.shape}")
print(f"Forma del conjunto de test: {X_test.shape}")
print(f"Número de clases: {len(newsgroups_train.target_names)}")
print(f"Clases: {newsgroups_train.target_names}")
print()

Forma del conjunto de entrenamiento: (11314, 101631)
Forma del conjunto de test: (7532, 101631)
Número de clases: 20
Clases: ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']



In [19]:
# Función para evaluar un modelo
def evaluate_model(model, X_train, y_train, X_test, y_test, model_name):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    f1 = f1_score(y_test, y_pred, average='macro')
    print(f"{model_name}: F1-score macro = {f1:.4f}")
    return f1, y_pred

print("EVALUACIÓN CON VECTORIZADOR BÁSICO TF-IDF:")
print("-" * 50)

# Evaluación inicial con MultinomialNB
multinomial_nb = MultinomialNB()
f1_multi_basic, _ = evaluate_model(multinomial_nb, X_train, y_train, X_test, y_test, "MultinomialNB (básico)")

# Evaluación inicial con ComplementNB
complement_nb = ComplementNB()
f1_comp_basic, _ = evaluate_model(complement_nb, X_train, y_train, X_test, y_test, "ComplementNB (básico)")

EVALUACIÓN CON VECTORIZADOR BÁSICO TF-IDF:
--------------------------------------------------
MultinomialNB (básico): F1-score macro = 0.5854
ComplementNB (básico): F1-score macro = 0.6930


In [20]:
# OPTIMIZACIÓN DE PARÁMETROS DEL VECTORIZADOR
print("OPTIMIZACIÓN DE PARÁMETROS DEL VECTORIZADOR:")
print("-" * 50)

# Lista de configuraciones de vectorizadores para probar
vectorizer_configs = [
    {
        'name': 'TF-IDF básico',
        'params': {}
    },
    {
        'name': 'TF-IDF con min_df=2',
        'params': {'min_df': 2}
    },
    {
        'name': 'TF-IDF con max_df=0.8',
        'params': {'max_df': 0.8}
    },
    {
        'name': 'TF-IDF con min_df=2, max_df=0.8',
        'params': {'min_df': 2, 'max_df': 0.8}
    },
    {
        'name': 'TF-IDF con max_features=10000',
        'params': {'max_features': 10000}
    },
    {
        'name': 'TF-IDF con ngram_range=(1,2)',
        'params': {'ngram_range': (1, 2), 'max_features': 20000}
    },
    {
        'name': 'TF-IDF con ngram_range=(1,2), min_df=2, max_df=0.8',
        'params': {'ngram_range': (1, 2), 'min_df': 2, 'max_df': 0.8, 'max_features': 15000}
    },
    {
        'name': 'TF-IDF con stop_words=english',
        'params': {'stop_words': 'english'}
    },
    {
        'name': 'TF-IDF optimizado',
        'params': {'min_df': 3, 'max_df': 0.7, 'ngram_range': (1, 2), 'max_features': 12000, 'stop_words': 'english'}
    }
]

# Lista de configuraciones de modelos para probar
model_configs = [
    {
        'name': 'MultinomialNB (alpha=1.0)',
        'model': MultinomialNB(alpha=1.0)
    },
    {
        'name': 'MultinomialNB (alpha=0.1)',
        'model': MultinomialNB(alpha=0.1)
    },
    {
        'name': 'MultinomialNB (alpha=0.01)',
        'model': MultinomialNB(alpha=0.01)
    },
    {
        'name': 'ComplementNB (alpha=1.0)',
        'model': ComplementNB(alpha=1.0)
    },
    {
        'name': 'ComplementNB (alpha=0.1)',
        'model': ComplementNB(alpha=0.1)
    },
    {
        'name': 'ComplementNB (alpha=0.01)',
        'model': ComplementNB(alpha=0.01)
    }
]

best_f1 = 0
best_config = None
results = []

# Probar todas las combinaciones
for vec_config in vectorizer_configs:
    print(f"\nProbando vectorizador: {vec_config['name']}")
    print("-" * 40)
    
    # Crear y entrenar el vectorizador
    vectorizer = TfidfVectorizer(**vec_config['params'])
    X_train_vec = vectorizer.fit_transform(newsgroups_train.data)
    X_test_vec = vectorizer.transform(newsgroups_test.data)
    
    print(f"Dimensionalidad: {X_train_vec.shape[1]} características")
    
    for model_config in model_configs:
        model = model_config['model']
        model_name = model_config['name']
        
        # Entrenar y evaluar
        f1, _ = evaluate_model(model, X_train_vec, y_train, X_test_vec, y_test, model_name)
        
        # Guardar resultado
        result = {
            'vectorizer': vec_config['name'],
            'model': model_name,
            'f1_score': f1,
            'vectorizer_params': vec_config['params'],
            'model_obj': model_config['model']
        }
        results.append(result)
        
        # Actualizar mejor resultado
        if f1 > best_f1:
            best_f1 = f1
            best_config = {
                'vectorizer': vectorizer,
                'model': model_config['model'],
                'vectorizer_name': vec_config['name'],
                'model_name': model_name,
                'f1_score': f1
            }

OPTIMIZACIÓN DE PARÁMETROS DEL VECTORIZADOR:
--------------------------------------------------

Probando vectorizador: TF-IDF básico
----------------------------------------
Dimensionalidad: 101631 características
MultinomialNB (alpha=1.0): F1-score macro = 0.5854
MultinomialNB (alpha=0.1): F1-score macro = 0.6565
MultinomialNB (alpha=0.01): F1-score macro = 0.6829
ComplementNB (alpha=1.0): F1-score macro = 0.6930
ComplementNB (alpha=0.1): F1-score macro = 0.6954
ComplementNB (alpha=0.01): F1-score macro = 0.6689

Probando vectorizador: TF-IDF con min_df=2
----------------------------------------
Dimensionalidad: 39423 características
MultinomialNB (alpha=1.0): F1-score macro = 0.5970
MultinomialNB (alpha=0.1): F1-score macro = 0.6727
MultinomialNB (alpha=0.01): F1-score macro = 0.6809
ComplementNB (alpha=1.0): F1-score macro = 0.6935
ComplementNB (alpha=0.1): F1-score macro = 0.6906
ComplementNB (alpha=0.01): F1-score macro = 0.6747

Probando vectorizador: TF-IDF con max_df=0.8
-----

In [21]:
print("\n" + "="*80)
print("RESUMEN DE RESULTADOS:")
print("="*80)

# Mostrar top 10 mejores resultados
results_sorted = sorted(results, key=lambda x: x['f1_score'], reverse=True)

print("TOP 10 MEJORES CONFIGURACIONES:")
print("-" * 50)
for i, result in enumerate(results_sorted[:10], 1):
    print(f"{i}. F1-score: {result['f1_score']:.4f}")
    print(f"   Vectorizador: {result['vectorizer']}")
    print(f"   Modelo: {result['model']}")
    print()

print("MEJOR CONFIGURACIÓN ENCONTRADA:")
print("-" * 40)
print(f"F1-score macro: {best_config['f1_score']:.4f}")
print(f"Vectorizador: {best_config['vectorizer_name']}")
print(f"Modelo: {best_config['model_name']}")
print()

# Entrenar el mejor modelo y mostrar métricas adicionales
print("ENTRENANDO MODELO FINAL CON LA MEJOR CONFIGURACIÓN:")
print("-" * 50)
best_model = best_config['model']
best_vectorizer = best_config['vectorizer']

# Re-entrenar con la mejor configuración
X_train_best = best_vectorizer.fit_transform(newsgroups_train.data)
X_test_best = best_vectorizer.transform(newsgroups_test.data)
best_model.fit(X_train_best, y_train)
y_pred_best = best_model.predict(X_test_best)

final_f1 = f1_score(y_test, y_pred_best, average='macro')
print(f"F1-score macro final: {final_f1:.4f}")
print(f"Número de características del mejor vectorizador: {X_train_best.shape[1]}")

# Análisis por clase
from sklearn.metrics import classification_report
print("\nREPORTE DE CLASIFICACIÓN DETALLADO:")
print("-" * 50)
print(classification_report(y_test, y_pred_best, target_names=newsgroups_train.target_names))


RESUMEN DE RESULTADOS:
TOP 10 MEJORES CONFIGURACIONES:
--------------------------------------------------
1. F1-score: 0.6954
   Vectorizador: TF-IDF básico
   Modelo: ComplementNB (alpha=0.1)

2. F1-score: 0.6950
   Vectorizador: TF-IDF con max_df=0.8
   Modelo: ComplementNB (alpha=0.1)

3. F1-score: 0.6936
   Vectorizador: TF-IDF con stop_words=english
   Modelo: ComplementNB (alpha=1.0)

4. F1-score: 0.6935
   Vectorizador: TF-IDF con min_df=2
   Modelo: ComplementNB (alpha=1.0)

5. F1-score: 0.6930
   Vectorizador: TF-IDF básico
   Modelo: ComplementNB (alpha=1.0)

6. F1-score: 0.6925
   Vectorizador: TF-IDF con min_df=2, max_df=0.8
   Modelo: ComplementNB (alpha=1.0)

7. F1-score: 0.6920
   Vectorizador: TF-IDF con max_df=0.8
   Modelo: ComplementNB (alpha=1.0)

8. F1-score: 0.6919
   Vectorizador: TF-IDF con stop_words=english
   Modelo: ComplementNB (alpha=0.1)

9. F1-score: 0.6907
   Vectorizador: TF-IDF con min_df=2, max_df=0.8
   Modelo: ComplementNB (alpha=0.1)

10. F1-scor

## Punto 3: 
  Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación (f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial y ComplementNB.

In [22]:

print("\n" + "="*100)
print("PUNTO 3: ANÁLISIS DE SIMILARIDAD ENTRE PALABRAS")
print("="*100)

# Usar el vectorizador original para obtener los nombres de las características
feature_names = tfidfvect.get_feature_names_out()
print(f"Número total de términos en el vocabulario: {len(feature_names)}")

# Transponer la matriz documento-término para obtener matriz término-documento
X_train_transposed = X_train.T  # Ahora cada fila es un término y cada columna es un documento
print(f"Forma de la matriz transpuesta (término-documento): {X_train_transposed.shape}")
print("Cada fila representa ahora un término/palabra y cada columna un documento\n")

# Seleccionar palabras manualmente para el análisis
# Elegimos palabras que sean interpretables y comunes en el dataset
selected_words = [
    'computer',    # Tecnología
    'government',  # Política  
    'medical',     # Medicina
    'religion',    # Religión
    'sports'       # Deportes
]

print("PALABRAS SELECCIONADAS PARA ANÁLISIS:")
print("-" * 50)
for word in selected_words:
    print(f"- {word}")
print()

# Función para encontrar el índice de una palabra en el vocabulario
def find_word_index(word, feature_names):
    try:
        return list(feature_names).index(word)
    except ValueError:
        return None

# Función para encontrar palabras similares basándose en documentos que contienen palabras similares
def find_similar_words_by_documents(word_index, X_transposed, feature_names, top_k=5):
    # Obtener el vector del término (fila correspondiente en la matriz transpuesta)
    word_vector = X_transposed[word_index]
    
    # Calcular similaridad coseno con todos los otros términos
    similarities = cosine_similarity(word_vector, X_transposed).flatten()
    
    # Obtener los índices de los términos más similares (excluyendo la palabra misma)
    # Tomamos top_k+1 porque el primero será la misma palabra
    most_similar_indices = similarities.argsort()[-(top_k+1):][::-1]
    
    # Remover la primera (la misma palabra)
    most_similar_indices = most_similar_indices[1:]
    
    return most_similar_indices, similarities

# Analizar cada palabra seleccionada
for word in selected_words:
    print(f"{'='*80}")
    print(f"ANÁLISIS DE SIMILARIDAD PARA: '{word}'")
    print(f"{'='*80}")
    
    # Encontrar el índice de la palabra
    word_index = find_word_index(word, feature_names)
    
    if word_index is None:
        print(f"La palabra '{word}' no se encuentra en el vocabulario.")
        print("Buscando palabras similares en el vocabulario...")
        
        # Buscar palabras que contengan la palabra como substring
        similar_in_vocab = [w for w in feature_names if word in w.lower()]
        if similar_in_vocab:
            print(f"Palabras relacionadas encontradas: {similar_in_vocab[:10]}")
            # Usar la primera palabra encontrada
            word = similar_in_vocab[0]
            word_index = find_word_index(word, feature_names)
            print(f"Usando '{word}' para el análisis.\n")
        else:
            print(f"No se encontraron palabras relacionadas con '{word}' en el vocabulario.\n")
            continue
    
    print(f"Índice de la palabra '{word}': {word_index}")
    
    # Obtener información sobre la frecuencia de la palabra
    word_vector = X_train_transposed[word_index]
    total_docs_with_word = (word_vector > 0).sum()
    max_tfidf = word_vector.max()
    
    print(f"Aparece en {total_docs_with_word} documentos")
    print(f"Valor TF-IDF máximo: {max_tfidf:.4f}")
    print()
    
    # Encontrar palabras similares
    similar_indices, similarities = find_similar_words_by_documents(
        word_index, X_train_transposed, feature_names, top_k=5
    )
    
    print(f"LAS 5 PALABRAS MÁS SIMILARES A '{word}':")
    print("-" * 50)
    
    for rank, similar_index in enumerate(similar_indices, 1):
        similar_word = feature_names[similar_index]
        similarity_score = similarities[similar_index]
        
        # Información adicional sobre la palabra similar
        similar_vector = X_train_transposed[similar_index]
        docs_with_similar = (similar_vector > 0).sum()
        max_tfidf_similar = similar_vector.max()
        
        print(f"{rank}. '{similar_word}'")
        print(f"   Similaridad coseno: {similarity_score:.4f}")
        print(f"   Aparece en: {docs_with_similar} documentos")
        print(f"   TF-IDF máximo: {max_tfidf_similar:.4f}")
        print()
    
    print()

# Análisis adicional: Matriz de similaridad entre las palabras seleccionadas
print("="*80)
print("MATRIZ DE SIMILARIDAD ENTRE LAS PALABRAS SELECCIONADAS")
print("="*80)

# Obtener los índices de todas las palabras que pudimos encontrar
valid_words = []
valid_indices = []

for word in selected_words:
    word_index = find_word_index(word, feature_names)
    if word_index is None:
        # Buscar alternativas
        similar_in_vocab = [w for w in feature_names if word in w.lower()]
        if similar_in_vocab:
            word = similar_in_vocab[0]
            word_index = find_word_index(word, feature_names)
    
    if word_index is not None:
        valid_words.append(word)
        valid_indices.append(word_index)

if len(valid_words) > 1:
    # Crear matriz de vectores de las palabras válidas
    word_vectors = X_train_transposed[valid_indices]
    
    # Calcular matriz de similaridad
    similarity_matrix = cosine_similarity(word_vectors)
    
    print("Matriz de similaridad coseno:")
    print("-" * 40)
    
    # Imprimir header
    print(f"{'':>12}", end="")
    for word in valid_words:
        print(f"{word:>12}", end="")
    print()
    
    # Imprimir matriz
    for i, word_i in enumerate(valid_words):
        print(f"{word_i:>12}", end="")
        for j, word_j in enumerate(valid_words):
            print(f"{similarity_matrix[i][j]:>12.4f}", end="")
        print()
    
    print()
    
    # Encontrar los pares más similares (excluyendo la diagonal)
    print("PARES DE PALABRAS MÁS SIMILARES:")
    print("-" * 40)
    
    pairs_similarity = []
    for i in range(len(valid_words)):
        for j in range(i+1, len(valid_words)):
            pairs_similarity.append({
                'word1': valid_words[i],
                'word2': valid_words[j], 
                'similarity': similarity_matrix[i][j]
            })
    
    # Ordenar por similaridad
    pairs_similarity.sort(key=lambda x: x['similarity'], reverse=True)
    
    for i, pair in enumerate(pairs_similarity[:3], 1):
        print(f"{i}. '{pair['word1']}' ↔ '{pair['word2']}': {pair['similarity']:.4f}")

else:
    print("No se encontraron suficientes palabras válidas para crear la matriz de similaridad.")

print("\n" + "="*80)
print("INTERPRETACIÓN DE LOS RESULTADOS:")
print("="*80)
print("""
Los vectores de palabras obtenidos mediante la transposición de la matriz TF-IDF
representan a cada palabra en el espacio de documentos. Esto significa que:

1. Dos palabras son similares si tienden a aparecer en documentos similares
2. La similaridad coseno mide qué tan frecuentemente las palabras co-ocurren
   en los mismos tipos de documentos
3. Palabras del mismo dominio temático (ej: tecnología, política) deberían
   mostrar mayor similaridad
4. Este enfoque captura relaciones semánticas basadas en el contexto de uso

Este método es una forma simple de obtener embeddings de palabras basados
en la distribución de documentos, similar en concepto a técnicas más
sofisticadas como Word2Vec o GloVe.
""")


PUNTO 3: ANÁLISIS DE SIMILARIDAD ENTRE PALABRAS
Número total de términos en el vocabulario: 101631
Forma de la matriz transpuesta (término-documento): (101631, 11314)
Cada fila representa ahora un término/palabra y cada columna un documento

PALABRAS SELECCIONADAS PARA ANÁLISIS:
--------------------------------------------------
- computer
- government
- medical
- religion
- sports

ANÁLISIS DE SIMILARIDAD PARA: 'computer'
Índice de la palabra 'computer': 28940
Aparece en 446 documentos
Valor TF-IDF máximo: 0.3716

LAS 5 PALABRAS MÁS SIMILARES A 'computer':
--------------------------------------------------
1. 'decwriter'
   Similaridad coseno: 0.1563
   Aparece en: 1 documentos
   TF-IDF máximo: 0.2512

2. 'deluged'
   Similaridad coseno: 0.1522
   Aparece en: 1 documentos
   TF-IDF máximo: 0.2446

3. 'harkens'
   Similaridad coseno: 0.1522
   Aparece en: 1 documentos
   TF-IDF máximo: 0.2446

4. 'shopper'
   Similaridad coseno: 0.1443
   Aparece en: 9 documentos
   TF-IDF máximo: 0.